In [1]:
import numpy as np
from skimage.measure import label
import nibabel as nb
import glob
import pandas as pd
from utils.surface_distance import compute_surface_distances, compute_surface_overlap_at_tolerance
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def getLargestCC(segmentation):
    labels = label(segmentation)
    assert( labels.max() != 0 ) # assume at least 1 CC
    largestCC = labels == np.argmax(np.bincount(labels.flat)[1:])+1
    return largestCC

def dice_surface_distance_perclass(vol_output, ground_truth, no_samples=40, mode='train'):
    surface_distance_perclass = np.zeros((3, 2))
    organ_tolerances = [1.0, 3.0, 3.0]
    if mode == 'train':
        pass
        #samples = np.random.choice(len(vol_output), no_samples)
        #print(vol_output.shape, ground_truth.shape)
        #vol_output, ground_truth = vol_output[samples], ground_truth[samples]
    for i in range(3):
        GT = (ground_truth == i)
        Pred = (vol_output == i)

        surface_dis = compute_surface_distances(GT, Pred, [2,2,3])
        avg_surface_distance = compute_surface_overlap_at_tolerance(surface_dis, organ_tolerances[i])
        surface_distance_perclass[i] = avg_surface_distance
    return surface_distance_perclass

In [3]:

def qc(data_nifty, samples_):
    vol_outs = []
    for s in samples_:
        vol_outs.append(nb.load(s).get_fdata())

    data = data_nifty.get_fdata()
    no_samples = data.shape[0]
    samples = np.random.choice(len(data), no_samples, replace=False)
    
    step = no_samples//len(samples_)+1
    counter = 0

    vol_outs = np.array(vol_outs)
    vol_out = np.zeros((no_samples, data.shape[-2], data.shape[-1]))
    for i in range(0, no_samples, step):
        vol_out[i:i+step] = vol_outs[counter, i:i+step]
        counter += 1
        
    all_data = data.copy()

    liver_seg =np.zeros_like(data)
    spleen_seg = np.zeros_like(data)
    
    cc = getLargestCC(data)
    liver_seg[cc] = 2

    data[cc] = 0
    ss = getLargestCC(data)
    spleen_seg[ss] = 1

    gt = liver_seg + spleen_seg
    gt = gt[np.sort(samples)]
    all_data = all_data[np.sort(samples)]
    out_sample_out = dice_surface_distance_perclass(vol_out, gt)
    out_all = dice_surface_distance_perclass(all_data, gt)
    return out_sample_out, out_all

In [ ]:
with open('/home/abhijit/Jyotirmay/my_thesis/dataset_groups/whole_body_datasets/KORA/test_volumes.txt', mode='r') as file_handler:
    volumes = file_handler.read().splitlines()


In [ ]:
# fb_k_data_nifty = nb.load('/home/abhijit/Jyotirmay/my_thesis/projects/full_bayesian/outs/full_bayesian_KORA_v2/full_bayesian_KORA_v2_predictions_KORA/KORA2452190_seg.nii.gz')
# fb_k_samples_ = glob.glob('/home/abhijit/Jyotirmay/my_thesis/projects/full_bayesian/outs/full_bayesian_KORA_v2/full_bayesian_KORA_v2_predictions_KORA/KORA2452190_samples/*')

# m_k_data_nifty = nb.load('/home/abhijit/Jyotirmay/my_thesis/projects/MC_dropout_quicknat/outs/MC_dropout_quicknat_KORA_v2/MC_dropout_quicknat_KORA_v2_predictions_KORA/KORA2452190_seg.nii.gz')
# m_k_samples_ = glob.glob('/home/abhijit/Jyotirmay/my_thesis/projects/MC_dropout_quicknat/outs/MC_dropout_quicknat_KORA_v2/MC_dropout_quicknat_KORA_v2_predictions_KORA/KORA2452190_samples/*')

# h_k_data_nifty = nb.load('/home/abhijit/Jyotirmay/my_thesis/projects/hierarchical_quicknat/outs/hierarchical_quicknat_KORA_v2/hierarchical_quicknat_KORA_v2_predictions_KORA/KORA2452190_seg.nii.gz')
# h_k_samples_ = glob.glob('/home/abhijit/Jyotirmay/my_thesis/projects/hierarchical_quicknat/outs/hierarchical_quicknat_KORA_v2/hierarchical_quicknat_KORA_v2_predictions_KORA/KORA2452190_samples/*')

# p_k_data_nifty = nb.load('/home/abhijit/Jyotirmay/my_thesis/projects/probabilistic_quicknat/outs/probabilistic_quicknat_KORA_v2/probabilistic_quicknat_KORA_v2_predictions_KORA/KORA2452190_seg.nii.gz')
# p_k_samples_ = glob.glob('/home/abhijit/Jyotirmay/my_thesis/projects/probabilistic_quicknat/outs/probabilistic_quicknat_KORA_v2/probabilistic_quicknat_KORA_v2_predictions_KORA/KORA2452190_samples/*')

projects = ['MC_dropout_quicknat', 'hierarchical_quicknat', 'probabilistic_quicknat']

dicts = {}
for p in projects:
    dicts[p] = {}
    for ijk, v in enumerate(volumes):
        print("\rProcessing {} {}/{}.".format(p, ijk, len(volumes)), end="")
        try:
            fb_k_data_nifty = nb.load(f'/home/abhijit/Jyotirmay/my_thesis/projects/{p}/outs/{p}_KORA_v2/{p}_KORA_v2_predictions_KORA/{v}_seg.nii.gz')
            fb_k_samples_ = glob.glob(f'/home/abhijit/Jyotirmay/my_thesis/projects/{p}/outs/{p}_KORA_v2/{p}_KORA_v2_predictions_KORA/{v}_samples/*')
            sample_out, all_data_out = qc(fb_k_data_nifty, fb_k_samples_)
            dicts[p][v] = sample_out[1:, 1:].reshape(-1,)
        except Exception as e:
            print(e)
            continue
    
    df = pd.DataFrame.from_dict(dicts[p], orient="index")
    df.to_csv(f'{p}_sample_out_qc.csv', index_label='volume_id')

Processing probabilistic_quicknat 116/153.

In [22]:
dd = {}
for ii, jj in dicts['full_bayesian'].items():
    print(jj)
    dd[ii] = jj[1:, 1:].reshape(-1,)

df = pd.DataFrame.from_dict(dd, orient="index")
df.to_csv(f'full_bayesian_sample_out_qc.csv', index_label='volume_id')

[[0.96714228 0.90158182]
 [0.99925981 0.82421263]
 [0.98933187 0.8451985 ]]
[[0.9777182  0.92227306]
 [0.99982933 0.92094274]
 [0.99586629 0.83398065]]
[[0.97426861 0.9241192 ]
 [0.98582525 0.89866191]
 [0.99771501 0.89600864]]
[[0.96252788 0.87276773]
 [0.99901148 0.71394435]
 [0.99621431 0.79276897]]
[[0.97033177 0.90791661]
 [0.99885658 0.91206389]
 [0.99854537 0.85941591]]
[[0.96372545 0.89162621]
 [0.99380137 0.88672775]
 [0.99081828 0.82237856]]
[[0.96872525 0.88953462]
 [0.99635309 0.91542045]
 [0.98610741 0.75416637]]
[[0.97866999 0.94106926]
 [0.99332127 0.90937745]
 [0.99670461 0.90510377]]
[[0.97338675 0.91728782]
 [0.99993296 0.87266328]
 [0.9925857  0.85527388]]
[[0.97050427 0.90672673]
 [0.99924466 0.88887856]
 [0.99068917 0.83034424]]
[[0.98220465 0.94545607]
 [0.99309216 0.96227425]
 [0.99914571 0.89049728]]
[[0.98021868 0.94319254]
 [0.99707492 0.82816951]
 [0.99575304 0.94276943]]
[[0.97145583 0.88784985]
 [0.97314208 0.74472636]
 [0.98565055 0.71049569]]
[[0.9651805 

In [ ]:
model_merged_feats_path = {'full_bayesian': './projects/full_bayesian/reports/full_bayesian_KORA_v2/KORA/10_1571866968.4002764_concat_report_final.csv',
    'full_bayesian_0dot01': './projects/full_bayesian/reports/full_bayesian_KORA_v4/KORA/10_1572514598.527084_concat_report_final.csv',
    'MC_dropout_quicknat': './projects/MC_dropout_quicknat/reports/MC_dropout_quicknat_KORA_v2/KORA/10_1572006141.7793334_concat_report_final.csv', 
    'probabilistic_quicknat': './projects/probabilistic_quicknat/reports/probabilistic_quicknat_KORA_v2/KORA/10_1571996796.7963011_concat_report_final.csv', 
    'hierarchical_quicknat': './projects/hierarchical_quicknat/reports/hierarchical_quicknat_KORA_v2/KORA/10_1571905560.9377904_concat_report_final.csv'}
model_merged_feats_path

In [ ]:
feats = [ 'iou_spleen', 'iou_liver', 'iou_mean', 'dice_spleen',
       'dice_liver', 'dice_mean', 'surface_distance_prtogt_spleen',
       'surface_distance_prtogt_liver', 'surface_distance_prtogt_mean',
       'surface_distance_gttopr_spleen', 'surface_distance_gttopr_liver',
       'surface_distance_gttopr_mean', 'surface_distance_avg_spleen',
       'surface_distance_avg_liver', 'surface_distance_avg_mean']

In [ ]:
for k, v in model_merged_feats_path.items():
    print(k)
    df = pd.read_csv(v)

    print(df[df.volume_id=="KORA2452190"][feats].T)
    print('')